In [1]:
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from urllib.request import urlopen
from json import loads
from functools import reduce

valid_years = ["2012", "2013", "2014", "2015", "2016"]

'''
Federal Information Processing Standards state code
'''
fips_mapping = { 1:"Alabama", 2:"Alaska",  3:"American Samoa", 4:"Arizona", 5:"Arkansas", 6:"California", 
                7:"Canal Zone", 8:"Colorado", 9:"Connecticut", 10:"Delaware", 11:"District of Columbia", 12:"Florida", 
                13:"Georgia", 14:"Guam", 15:"Hawaii", 16:"Idaho", 17:"Illinois", 18:"Indiana", 19:"Iowa", 20:"Kansas",
                21:"Kentucky", 22:"Louisiana", 23:"Maine", 24:"Maryland", 25:"Massachusetts", 26:"Michigan", 
                27:"Minnesota", 28:"Mississippi", 29:"Missouri", 30:"Montana", 31:"Nebraska", 32:"Nevada", 
                33:"New Hampshire", 34:"New Jersey", 35:"New Mexico", 36:"New York", 37:"North Carolina", 
                38:"North Dakota", 39:"Ohio", 40:"Oklahoma", 41:"Oregon", 42:"Pennsylvania", 43:"Puerto Rico", 
                44:"Rhode Island", 45:"South Carolina", 46:"South Dakota", 47:"Tennessee", 48:"Texas", 49:"Utah", 
                50:"Vermont", 51:"Virginia", 52:"Virgin Islands of the US", 53:"Washington", 54:"West Virginia", 
                55:"Wisconsin", 56:"Wyoming", 58:"Department of Defense Dependent Schools (overseas)", 
                59:"Bureau of Indian Education", 60:"American Samoa", 61:"Department of Defense Dependent Schools (domestic)", 
                63:"Department of Defense Education Activity", 64:"Federated States of Micronesia", 
                65:"Mariana Islands waters (including Guam)", 66:"Guam", 67:"Johnston Atoll", 68:"Marshall Islands", 
                69:"Northern Mariana Islands", 70:"Palau", 71:"Midway Islands", 72:"Puerto Rico", 
                74:"US Minor Outlying Islands", 75:"Atlantic coast from North Carolina to Florida, and the coasts of Puerto Rico and Virgin Islands", 
                76:"Navassa Island", 78:"Virgin Islands of the US", 79:"Wake Island", 81:"Baker Island", 84:"Howland Island", 
                86:"Jarvis Island", 89:"Kingman Reef", 95:"Palmyra Atoll", -1:"Missing/not reported", -2:"Not applicable", -3:"Suppressed data"
               }
'''
Method accepts array of urls
Returns concatenated dataframe with data from urls
''' 
def get_data_frames(urls):  
    dfs = []   
    for url in urls:
        count = 0
#     while url:
#Use the following while loop so I can fetch less pages until I can make this method async:
        while url and count <= 3:
            print("Fetching data from", url)
            response = urlopen(url)
            data = loads(response.read())
            dfs.append(pd.DataFrame.from_dict(data["results"]))
            url = data["next"]
            print("Next url:", url)
            count+=1
    return pd.concat(dfs, ignore_index=True, sort=True)

In [ ]:
'''
Unforunately, API does not have years past 2016.
Following loads district level financial data w/ "leaid": 
Local education agency identification number (NCES) as a common identifier.
'''
finance_urls = ["https://educationdata.urban.org/api/v1/school-districts/ccd/finance/{}/".format(yr) for yr in valid_years]
finance_df = get_data_frames(finance_urls)

Fetching data from https://educationdata.urban.org/api/v1/school-districts/ccd/finance/2012/
Next url: https://educationdata.urban.org/api/v1/school-districts/ccd/finance/2012/?page=2
Fetching data from https://educationdata.urban.org/api/v1/school-districts/ccd/finance/2012/?page=2
Next url: https://educationdata.urban.org/api/v1/school-districts/ccd/finance/2012/?page=3
Fetching data from https://educationdata.urban.org/api/v1/school-districts/ccd/finance/2012/?page=3
Next url: https://educationdata.urban.org/api/v1/school-districts/ccd/finance/2012/?page=4
Fetching data from https://educationdata.urban.org/api/v1/school-districts/ccd/finance/2012/?page=4
Next url: https://educationdata.urban.org/api/v1/school-districts/ccd/finance/2012/?page=5
Fetching data from https://educationdata.urban.org/api/v1/school-districts/ccd/finance/2013/
Next url: https://educationdata.urban.org/api/v1/school-districts/ccd/finance/2013/?page=2
Fetching data from https://educationdata.urban.org/api/v1/s

In [ ]:
finance_df.head()

In [ ]:
'''
Unforunately, API does not have years past 2016.
Following loads district level student poverty data w/ "leaid": 
Local education agency identification number (NCES) as a common identifier.
'''

poverty_urls = ["https://educationdata.urban.org/api/v1/school-districts/saipe/{}/".format(yr) for yr in valid_years]
poverty_df = get_data_frames(finance_urls)

In [ ]:
poverty_df.head()

In [ ]:
'''
Unforunately, API does not have years past 2016.
And states: ['AL' 'AK' 'AZ' 'IL' 'CA' 'MI' 'MD' 'MN' 'TX' 'AR' 'NV']
Following loads geographic data w/ "leaid": 
Local education agency identification number (NCES) as a common identifier.
'''

geographic_urls = ["https://educationdata.urban.org/api/v1/school-districts/ccd/directory/{}/".format(yr) for yr in valid_years]
geographic_df = get_data_frames(finance_urls)

In [ ]:
geographic_df.head()

In [ ]:
df = pd.merge(finance_df, geographic_df, on=["leaid"])
df.head()

In [ ]:
'''
Calculate total revenue 'rev_total' (fed, state, local) for all states.
'''
df.rename(columns={'fips_x':'State'}, inplace = True)
df.rename(columns={'year_y':'Year'}, inplace = True)

grouped_series = df.groupby(['State', 'Year'])['rev_total_x'].sum().reset_index(name='Total Revenue')

# num_years = len(grouped_series['Year'].unique())
state_codes = grouped_series['State'].unique()
years = grouped_series['Year'].unique()

colors = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(len(grouped_series['State'].unique()))]

'''
Generate sample graphs using colors and random ints
'''

# Sample Graph 1
for i in range(len(state_codes)):
    plt.scatter(random.randint(0, 10), random.randint(0,10), c=colors[i], s=200)

plt.title('Sample Graph 1')
plt.show()
    
# Sample Graph 2  
fig, ax = plt.subplots()
for i in range(len(state_codes)):
    x, y = np.random.rand(2, len(years))
    scale = 200.0 * np.random.rand(len(years))
    ax.scatter(x, y, c=colors[i], s=scale, label=state_codes[i],
               alpha=1, edgecolors='none')

ax.legend(title='States')
ax.grid(True)

plt.title('Sample Graph 2')
plt.show()

In [ ]:
'''
Plot the total revenue 'rev_total' (fed, state, local) for all states by year
'''
fig, ax = plt.subplots()
for i in range(len(state_codes)):
    curr_state_code = state_codes[i]
    curr_total_revenues = grouped_series.loc[grouped_series['State'] == curr_state_code]['Total Revenue']/1000000
    
    if(len(curr_total_revenues) != len(years)):
        print("Missing some data for state: ", fips_mapping[curr_state_code], " . Skipping...")
    else:
        x , y =  years, curr_total_revenues
        ax.scatter(x, y, c=colors[i], label=fips_mapping[curr_state_code], alpha=1, edgecolors='none')

ax.legend(title='States', bbox_to_anchor=(1.0, 1.0))
ax.grid(True)

plt.title('Revue by State Over Time')
plt.show()

In [ ]:
'''
Plot the total revenue 'rev_total' (fed, state, local) for all states by year starting at origin
'''
fig, ax = plt.subplots()

min_revenue = float('inf')
max_revenue = -1 * float('inf')

for i in range(len(state_codes)):
    curr_state_code = state_codes[i]
    curr_total_revenues = grouped_series.loc[grouped_series['State']== curr_state_code]['Total Revenue']
    
    curr_min = np.min(curr_total_revenues)
    revenue_normalizer = lambda t: (t - curr_min)/1000000
    normalized_revenues = np.array([revenue_normalizer(x) for x in curr_total_revenues])
    
    if(len(curr_total_revenues) != len(years)):
        print("Missing some data for state: ", fips_mapping[curr_state_code], " . Skipping...")
    else:
        x , y =  years, normalized_revenues
        plt.scatter(x, y, c=colors[i], label=fips_mapping[curr_state_code], alpha=1, edgecolors='none')
        plt.plot(x, y)

ax.legend(title='States', bbox_to_anchor=(1, 1))
ax.grid(True)

plt.title('Revenue By State Over Time Normalised from $0')
plt.xlabel('Years', fontsize=12)
plt.ylabel('Normalized Total Revenue($M)', fontsize=12)
plt.xticks(np.arange(np.min(years), np.max(years)+1, 1))

plt.show()